In [2]:
import os
import csv
# import umap
import json
import librosa
import numpy as np
import matplotlib.pyplot as plt
# from sklearn import preprocessing
# from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import MinMaxScaler
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen

In [10]:
import file_load
video_path = "./dataset"
name = ".mp3"

file_list, file_name, folder_list, folder_name = file_load.file_path(video_path, name)

for mp3, name in zip(file_list, file_name):
    save_path = mp3.replace('.mp3','')
    print(save_path)
    wave, sr = librosa.load(mp3,sr = 16000)
    audio_size = np.size(wave)
    print(wave.shape)
    split_size = 480000
    count = 0
    for split_point in range(0, audio_size, split_size):
        if count<1:
            split_audio = wave[split_point : split_point + split_size]
            np.save(save_path+'_{}'.format(count), split_audio)
            print("wave split {} OK".format(split_audio.shape))
        count += 1

[ 3.71181071e-01  1.10457182e+00  2.69314855e-01  7.34921813e-01
  2.22904533e-01  1.79228380e-01  4.18587208e-01  4.45207715e-01
  6.09436035e-01  5.46478331e-01  1.88525423e-01  4.14474994e-01
  4.69632179e-01  7.70063519e-01  8.50444078e-01  4.05170649e-01
 -3.90764117e+00  1.49816170e+01 -1.18414581e+00  6.95862341e+00
  2.33373141e+00 -4.38541174e-01 -1.90898442e+00 -1.20004082e+00
  6.66624403e+00  5.27432871e+00 -5.89319229e-01  4.42398882e+00
 -2.40421295e+00 -7.90361691e+00  1.08534164e+01  1.16475582e+00
  1.04082673e-03 -8.27546720e-03  4.48830156e-04 -5.32444606e-03
 -3.11489425e-03 -1.25014478e-03  2.22794867e-03 -1.32472962e-03
 -3.51779141e-03 -1.64213364e-03  3.81496097e-05 -3.81557825e-03
  6.59245926e-04  5.00773977e-03 -4.91890775e-03 -1.83158922e-03]


In [9]:
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen
import file_load

model_path = './wavenet-ckpt/wavenet-ckpt/model.ckpt-200000' #模型位置
npy_path = './dataset'#dataset位置
name = '.npy' #欲讀取的檔名

def wavenet_encode(file_path):
    audio = np.load(file_path)
    encoding = fastgen.encode(audio, model_path, len(audio))
    print(encoding.reshape((-1, 16)).shape)
    return encoding.reshape((-1, 16))

file_list, file_name, folder_list, folder_name = file_load.file_path(npy_path, name)

for npy, name in zip(file_list, file_name):
    print(npy)
    wavenet_data = wavenet_encode(npy)
    std_wavenet = np.std(wavenet_data, axis=0)
    mean_wavenet = np.mean(wavenet_data, axis=0)
    
    average_difference_channels = np.zeros((16,))
    
    for i in range(0, len(wavenet_data) - 2, 2):
        temp = wavenet_data[i] - wavenet_data[i+1]
        average_difference_channels += temp
    average_difference_channels /= (len(wavenet_data) // 2)   
    average_difference_channels = np.array(average_difference_channels)
    
    concat_features_wavenet = np.hstack((std_wavenet, mean_wavenet))
    concat_features_wavenet = np.hstack((concat_features_wavenet, average_difference_channels))
    
    save_path = npy.replace('.npy', "") 
    print('save',concat_features_wavenet.shape)
    np.save(save_path+'_OK',concat_features_wavenet) #存特徵向量

INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
save (48,)
./dataset/no8/Old School Turntables
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000


KeyboardInterrupt: 

In [11]:
import file_load
import numpy as np
npy_path = './dataset/'#dataset位置
name = 'OK.npy' #欲讀取的檔名

file_list, file_name, folder_list, folder_name = file_load.file_path(npy_path, name)

for c in range(len(folder_name)):
    fi_list, fi_name, fo_list, fo_name = file_load.file_path(npy_path+'no'+str(c+1), name)
    npy_list = []
    name_list = []
    print('no'+str(c+1))
    print('')
    
    for npy, n in zip(fi_list, fi_name):
        vec = np.load(npy)
        npy_list.append(vec)
        name_list.append(n.replace('_0_OK', ''))
        print(n.replace('_0_OK', ''))
        
    if len(npy_list)==3:
        print('')
        dist = np.linalg.norm(npy_list[0]-npy_list[1])
        print(dist,'    {} 到 {}'.format(name_list[0], name_list[1]))
        dist = np.linalg.norm(npy_list[0]-npy_list[2])
        print(dist,'    {} 到 {}'.format(name_list[0], name_list[2]))
        dist = np.linalg.norm(npy_list[1]-npy_list[2])
        print(dist,'    {} 到 {}'.format(name_list[1], name_list[2]))
    
    elif len(npy_list)==2:
        print('')
        dist = np.linalg.norm(npy_list[0]-npy_list[1])
        print(dist)
    print("================")

no1

YOUTH
Bloom
MYMYMY

2.830699914456322     YOUTH 到 Bloom
2.27186095339642     YOUTH 到 MYMYMY
0.8675373922773715     Bloom 到 MYMYMY
no2

RainySeason
ILoveToo
Decalcomanie

1.3239942531157127     RainySeason 到 ILoveToo
1.8122381616397454     RainySeason 到 Decalcomanie
2.65248001066036     ILoveToo 到 Decalcomanie
no3

gogobebe
Egotistic
Angel

0.48026388253221775     gogobebe 到 Egotistic
1.788494752006492     gogobebe 到 Angel
1.5435984406194327     Egotistic 到 Angel
no4

SF
Lie
Hollywood

1.0652123327911913     SF 到 Lie
1.041392130835179     SF 到 Hollywood
0.6575773697672002     Lie 到 Hollywood
no5

EpikHigh-Decalcomanie
Colde-Lovestruck
EpikHigh-NOTHANXXX

1.7957430801751673     EpikHigh-Decalcomanie 到 Colde-Lovestruck
0.7965891389392296     EpikHigh-Decalcomanie 到 EpikHigh-NOTHANXXX
2.0336525322391497     Colde-Lovestruck 到 EpikHigh-NOTHANXXX
no6

Wolves
SidetoSide
petedavidson

1.6356742053229116     Wolves 到 SidetoSide
2.803708321277067     Wolves 到 petedavidson
1.2793955245668986

In [12]:
import numpy as np 
p = './dataset/no1/Bloom_0_OK.npy'
np.load(p)

array([ 4.59068298e-01,  1.62310767e+00,  2.50073433e-01,  1.02322268e+00,
        2.09206596e-01,  1.61392853e-01,  3.11768711e-01,  6.82018816e-01,
        8.68239224e-01,  7.98025846e-01,  1.44931659e-01,  2.83834904e-01,
        2.95970440e-01,  1.18260157e+00,  1.22722971e+00,  3.89136642e-01,
       -3.96077871e+00,  1.59558716e+01, -1.24136865e+00,  7.63066673e+00,
        2.46815872e+00, -7.55068898e-01, -1.30806649e+00, -1.98387301e+00,
        7.19348097e+00,  5.80535650e+00, -7.43454039e-01,  4.27498531e+00,
       -1.92947400e+00, -8.78628159e+00,  1.14011850e+01,  1.62685990e+00,
        1.23494762e-03,  1.55547338e-03, -1.39444799e-03,  1.79769850e-03,
       -1.13682385e-03,  1.23434775e-03,  2.88188257e-03, -2.13836032e-03,
        4.01615078e-04,  2.20636985e-03, -9.51589109e-04, -9.38862307e-04,
        2.30251788e-03, -1.45582664e-03, -4.53911378e-03,  1.89560945e-03])